<a href="https://colab.research.google.com/github/yudumpacin/NLPStudyNotes/blob/main/Turkish_LLM_Experiments_Trendyol_7B_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install accelerate

In [3]:
pip install bitsandbytes

In [4]:
from transformers import AutoModelForCausalLM, LlamaTokenizer, pipeline

model_id = "Trendyol/Trendyol-LLM-7b-chat-v0.1"
tokenizer = LlamaTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             device_map='auto',
                                             load_in_8bit=True)

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [5]:
pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                device_map="auto",
                max_new_tokens=1024,
                return_full_text=True,
                repetition_penalty=1.1
               )

In [6]:
DEFAULT_SYSTEM_PROMPT = "Sen yardımcı bir asistansın ve sana verilen talimatlar doğrultusunda en iyi cevabı üretmeye çalışacaksın.\n"

In [7]:
TEMPLATE = (
    "[INST] <<SYS>>\n"
    "{system_prompt}\n"
    "<</SYS>>\n\n"
    "{instruction} [/INST]"
)

In [8]:
def generate_prompt(instruction, system_prompt=DEFAULT_SYSTEM_PROMPT):
    return TEMPLATE.format_map({'instruction': instruction,'system_prompt': system_prompt})

sampling_params = dict(do_sample=True, temperature=0.3, top_k=50, top_p=0.9)

def generate_output(user_query, sys_prompt=DEFAULT_SYSTEM_PROMPT):
    prompt = generate_prompt(user_query, sys_prompt)
    outputs = pipe(prompt,
               **sampling_params
              )
    return outputs[0]["generated_text"].split("[/INST]")[-1]


In [9]:
eval_list = []

In [10]:
def ask_question(query):
  evals = {}
  user_query = query
  response = generate_output(user_query)
  print(response)
  evals["question"] = query
  evals["response"] = response
  eval_list.append(evals)


In [11]:
ask_question("Türkiye'de kaç il var?")

  Türkiye'de 78 il bulunmaktadır.


In [12]:
ask_question("Dede Korkut hikayelerinden birini anlat")

  Dede Korkut hikayeleri arasında en bilinenlerden biri "Deli Dumrul"dur. Hikaye, iki kardeşin, Deli Dumrul ve Köroğlu'nun, birbirlerini öldürmek için yarıştıkları bir mücadeleyi anlatır. Deli Dumrul, Köroğlu'nu yener ve onun başını keserek zafer kazanır. Ancak, Köroğlu'nun kardeşi de Deli Dumrul'u öldürür ve bu şekilde hikaye sona erer.


In [ ]:
question = """Aşağıdaki metne göre Mercidabık savaşı nerede gerçekleşmiştir? Metin: Mercidâbık Muharebesi, 24 Ağustos 1516 tarihinde I. Selim komutasındaki Osmanlı Devleti ordusu ile
 Kansu Gavri komutasındaki Memlûk Devleti ordusu arasında gerçekleşen muharebedir. Suriye'nin Halep şehrinin kuzeyinde gerçekleşen muharebe, Osmanlı ordusunun mutlak zaferiyle
 sonuçlanmıştır. Muharebenin sonucunda Memlûk Sultanı Kansu Gavri ölmüş, Memlûk kuvvetleri bozulmuş ve Suriye toprakları Osmanlı Devleti egemenliğine açılmıştır. Jane Hathaway'ın
  yazdığına göre Kansu Gavri'nin cesedi hiç bulunamadı ve çok geçmeden, onun cinler tarafından alınıp kaçırıldığına dair rivayetler yayıldı.[11] Memlük İmparatorluğu, kuruluşundan
  beri, tarihi boyunca böyle bir meydan muharebesi kaybetmemiş ve hiçbir hükümdarını muharebe meydanında bırakmamıştır. Moğollar, İlhanlılar, Timur bile Memlükler'i böylesine bir
  mağlubiyete uğratamamışlardır. Bu bozgundan sonra Mısır'ı değilse bile, Suriye, Lübnan, Filistin, hatta Hicaz'ı Osmanlılar'ın eline düşmekten koruyabilecek hiçbir kuvvet
  kalmamıştır."""

ask_question(question)


In [ ]:
ask_question("Mercidabık savaşı nerede gerçekleşmiştir?")

In [ ]:
ask_question("2023 yılında Türkiye'de gerçekleşen en önemli olayları tarihleri ile birlikte ver")

In [ ]:
ask_question("""Bir çiftçinin bahçesinde tavuklar ve inekler var. Çiftçi, hayvanların toplam sayısını saydığında 30 adet hayvan olduğunu görüyor.
Ancak, hayvanların toplam bacak sayısını saydığında 100 bacak olduğunu fark ediyor.
 Çiftçinin kaç tavuğu ve kaç ineği var?""")

In [ ]:
ask_question("Trendyol ile ilgili bir şiir yazar mısın?")

In [ ]:
ask_question("Türkiye'nin en iyi e-ticaret sitesi nedir?")


In [20]:
import pandas as pd

In [21]:
pd.DataFrame(eval_list)

,question,response
0,Türkiye'de kaç il var?,Türkiye'de 78 il bulunmaktadır.
1,Dede Korkut hikayelerinden birini anlat,Dede Korkut hikayeleri arasında en bilinenle...
2,Aşağıdaki metne göre Mercidabık savaşı nerede ...,"Metinde belirtilen bilgilere göre, Mercidabı..."
3,Mercidabık savaşı nerede gerçekleşmiştir?,"Mercidabık Savaşı, 1516 yılında Suriye'nin k..."
4,2023 yılında Türkiye'de gerçekleşen en önemli ...,- 19 Ekim 2023: Cumhurbaşkanı Recep Tayyip E...
5,Bir çiftçinin bahçesinde tavuklar ve inekler v...,Çiftçinin 10 tavuk ve 20 ineği vardır.
6,Trendyol ile ilgili bir şiir yazar mısın?,"Trendyol'un mağazası, her zaman alışveriş iç..."
7,Türkiye'nin en iyi e-ticaret sitesi nedir?,Türkiye'nin en iyi e-ticaret sitesi Hepsibur...
